### Dupla:
    Gilgleison Paulino Lima, M:378571
    Anny Caroline da Silva Cruz, M:378563

## Exercício - Titanic
disponível em https://www.kaggle.com/c/titanic/overview

#### Descrição
O naufrágio do RMS Titanic é um dos naufrágios mais famosos da história. Em 15 de abril de 1912, durante sua viagem inaugural, o Titanic afundou após colidir com um iceberg, matando 1502 de 2224 passageiros e tripulantes. Essa tragédia chocou a comunidade internacional e levou a melhores regulamentações de segurança para os navios.

Uma das razões pelas quais o naufrágio levou a tal perda de vidas foi que não havia botes salva-vidas suficientes para os passageiros e a tripulação. Embora houvesse algum elemento de sorte envolvido na sobrevivência do naufrágio, alguns grupos de pessoas eram mais propensos a sobreviver do que outros, como mulheres, crianças e a classe alta.

Nesse desafio, a atividade é que você conclua a análise de que tipo de pessoa provavelmente sobreviverá. Em particular, solicitamos você deve aplicar classificadores de árvores de decisão para esta atividade.


### Descrição do dataset
Os dados foram divididos em dois grupos:

conjunto de treinamento (train.csv)
conjunto de teste (test.csv)
O conjunto de treinamento deve ser usado para criar seus modelos de aprendizado de máquina. Para o conjunto de treinamento, fornecemos o resultado (também conhecido como "verdade do solo") para cada passageiro. Seu modelo será baseado em "atributos", como sexo e classe dos passageiros. Você também pode usar a engenharia de atributos para criar novos atributos.

Parte do conjunto de treino deve ser usado para ver o desempenho do seu modelo em dados não vistos. 

Para o conjunto de testes, não é fornecido o rótulo de classe para cada passageiro. É seu trabalho prever esses resultados. Para cada passageiro no conjunto de testes, use o modelo que você treinou para prever se eles sobreviveram ou não ao naufrágio do Titanic. O resultado deve ser submetido na plataforma Kaggle (opção **submit predictions**) a fim de calcular a acurácia. 

Também incluímos gender_submission.csv, um conjunto de previsões que supõem que todas e apenas passageiras sobrevivem, como um exemplo da aparência de um arquivo de envio.

### Dicionário de Dados

**Variable -	Definition	- Key**

survival - Survival	- 0 = No, 1 = Yes

pclass -	Ticket class -	1 = 1st, 2 = 2nd, 3 = 3rd

sex -	Sex	

Age	- Age in years	

sibsp -	# of siblings / spouses aboard the Titanic	

parch- # of parents / children aboard the Titanic	

ticket - Ticket number	

fare - Passenger fare	

cabin- Cabin number	

embarked- Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton

### Carregando os dados

In [165]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split

In [166]:
data_train = pd.read_csv("titanic/train.csv", na_values='?', sep=',')
data_test = pd.read_csv("titanic/test.csv", na_values='?')

In [167]:
data_train.shape, data_test.shape

((891, 12), (418, 11))

In [168]:
data_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [169]:
data_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [170]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [171]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


#### Tranformando a coluna passengerId no index

In [172]:
data_train.set_index('PassengerId', drop=True, inplace=True)

In [173]:
data_test.set_index('PassengerId', drop=True, inplace=True)

#### Dropando colunas irrelevantes ao problema

In [174]:
#dataset de treino
data_train.drop('Fare', axis=1,inplace=True)
data_train.drop('Ticket', axis=1,inplace=True)
data_train.drop('Name', axis=1,inplace=True)
#dataset de test
data_test.drop('Fare', axis=1,inplace=True)
data_test.drop('Ticket', axis=1,inplace=True)
data_test.drop('Name', axis=1,inplace=True)

#### Limpando os dados nulos dos datasets

In [175]:
#caso a coluna tenha mais de 50% de dados nulos ela será dropada, 
#nesse caso a coluna cabin será dropada, pois tem muitos valores faltantes
data_train.dropna(axis=1, thresh=(data_train.shape[0] * 0.5), inplace=True) 
data_test.dropna(axis=1, thresh=(data_test.shape[0] * 0.5), inplace=True)

In [176]:
data_train.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Embarked      2
dtype: int64

In [177]:
data_test.isna().sum()

Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Embarked     0
dtype: int64

In [178]:
#substituindo os valores faltantes nas colunas Age e Embarked do dataset de treino
data_train.replace(np.nan, data_train['Age'].mean(), inplace=True)
data_train.replace(np.nan, data_train['Embarked'].mode(), inplace=True)
#substituindo os valores faltantes na coluna Age dataset de test
data_test.replace(np.nan, data_test['Age'].mean(), inplace=True)

#### Substituindo valores categóricos

In [179]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 7 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Embarked    891 non-null object
dtypes: float64(1), int64(4), object(2)
memory usage: 55.7+ KB


In [180]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 6 columns):
Pclass      418 non-null int64
Sex         418 non-null object
Age         418 non-null float64
SibSp       418 non-null int64
Parch       418 non-null int64
Embarked    418 non-null object
dtypes: float64(1), int64(3), object(2)
memory usage: 22.9+ KB


In [181]:
#dados de treino
data_train['Sex'] = pd.Categorical(data_train['Sex']).codes
data_train['Embarked'] = pd.Categorical(data_train['Embarked']).codes
#dados de teste
data_test['Sex'] = pd.Categorical(data_test['Sex']).codes
data_test['Embarked'] = pd.Categorical(data_test['Embarked']).codes

In [182]:
data_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
PassengerId,,,,,,,
1,0,3,1,22.0,1,0,3
2,1,1,0,38.0,1,0,1
3,1,3,0,26.0,0,0,3
4,1,1,0,35.0,1,0,3
5,0,3,1,35.0,0,0,3


In [183]:
data_test.head()

,Pclass,Sex,Age,SibSp,Parch,Embarked
PassengerId,,,,,,
892,3,1,34.5,0,0,1
893,3,0,47.0,1,0,2
894,2,1,62.0,0,0,1
895,3,1,27.0,0,0,2
896,3,0,22.0,1,1,2


#### Dividindo o dataset de treino em treino e validação

In [184]:
y = data_train['Survived'].values
data_train.drop('Survived', axis=1, inplace=True)

In [185]:
x = data_train.values

In [186]:
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.10, random_state=42)

#### Treinando o modelo

In [187]:
dt = DecisionTreeClassifier(max_depth=5, random_state=42)
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [188]:
print(classification_report(dt.predict(X_val), y_val))

              precision    recall  f1-score   support

           0       0.89      0.86      0.87        56
           1       0.78      0.82      0.80        34

    accuracy                           0.84        90
   macro avg       0.83      0.84      0.84        90
weighted avg       0.85      0.84      0.85        90



##### Treinando um novo modelo apenas com as features mais importates

In [189]:
dft = dt.feature_importances_ > 0.01

In [190]:
dt3 = DecisionTreeClassifier(max_depth=3, random_state=42).fit(X_train[:,dft], y_train)

In [191]:
print(classification_report(dt3.predict(X_val[:,dft]), y_val))

              precision    recall  f1-score   support

           0       0.91      0.83      0.87        59
           1       0.72      0.84      0.78        31

    accuracy                           0.83        90
   macro avg       0.81      0.83      0.82        90
weighted avg       0.84      0.83      0.84        90



In [192]:
dt5 = DecisionTreeClassifier(max_depth=5, random_state=42).fit(X_train[:,dft], y_train)

In [193]:
print(classification_report(dt5.predict(X_val[:,dft]), y_val))

              precision    recall  f1-score   support

           0       0.89      0.86      0.87        56
           1       0.78      0.82      0.80        34

    accuracy                           0.84        90
   macro avg       0.83      0.84      0.84        90
weighted avg       0.85      0.84      0.85        90



#### Gerando as labels para os dados de teste

In [194]:
x_test = data_test.values

In [195]:
#será usado a arvore com max_depth = 5, pois teve um melhor resultado.
y_pred_test = dt5.predict(x_test[:,dft])

In [196]:
y_pred_test

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,